In [1]:
import pandas as pd
df = pd.read_csv(r"https://raw.githubusercontent.com/microbhai/CustomerChurnAnalysis/master/telecommunication_customer_churn.csv")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CaseOrder             10000 non-null  int64  
 1   Customer_id           10000 non-null  object 
 2   Interaction           10000 non-null  object 
 3   UID                   10000 non-null  object 
 4   City                  10000 non-null  object 
 5   State                 10000 non-null  object 
 6   County                10000 non-null  object 
 7   Zip                   10000 non-null  int64  
 8   Lat                   10000 non-null  float64
 9   Lng                   10000 non-null  float64
 10  Population            10000 non-null  int64  
 11  Area                  10000 non-null  object 
 12  TimeZone              10000 non-null  object 
 13  Job                   10000 non-null  object 
 14  Children              10000 non-null  int64  
 15  Age                 

In [3]:
df.Churn.head(10)

0     No
1    Yes
2     No
3     No
4    Yes
5     No
6    Yes
7    Yes
8     No
9     No
Name: Churn, dtype: object

In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


In [4]:
X = df.drop(['CaseOrder','Customer_id','Interaction','UID','City','State','Zip','Lat','Contacts','Tenure','Lng','Population','TimeZone','Churn','Email',], axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   County                10000 non-null  object 
 1   Area                  10000 non-null  object 
 2   Job                   10000 non-null  object 
 3   Children              10000 non-null  int64  
 4   Age                   10000 non-null  int64  
 5   Income                10000 non-null  float64
 6   Marital               10000 non-null  object 
 7   Gender                10000 non-null  object 
 8   Outage_sec_perweek    10000 non-null  float64
 9   Yearly_equip_failure  10000 non-null  int64  
 10  Techie                10000 non-null  object 
 11  Contract              10000 non-null  object 
 12  Port_modem            10000 non-null  object 
 13  Tablet                10000 non-null  object 
 14  InternetService       7871 non-null   object 
 15  Phone               

In [10]:
#Let us define the response variable
y = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
y = y.values.reshape(-1, 1)
y_scaled = StandardScaler().fit_transform(y)

#Now, Let us define the X independent variable; which has a mixture of both categorical and numerical variables
#We separate the variables accordingly
categorical_features = X[['County', 'Area', 'Job', 'Marital', 'Gender', 'Techie', 'Contract', 'Port_modem', 'Tablet', 'InternetService', 'Phone', 'Multiple', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'PaymentMethod']]
numeric_features = X[['Children', 'Age', 'Income', 'Outage_sec_perweek', 'Yearly_equip_failure', 'MonthlyCharge', 'Bandwidth_GB_Year']]

encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = encoder.fit_transform(categorical_features).toarray()

X_final = np.concatenate([X_encoded, numeric_features.values], axis=1)
X = pd.DataFrame(X_final)

X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

model = sm.Logit(y_train, X_train).fit()
print(model.summary())


KeyError: "None of [Index(['County', 'Area', 'Job', 'Marital', 'Gender', 'Techie', 'Contract',\n       'Port_modem', 'Tablet', 'InternetService', 'Phone', 'Multiple',\n       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',\n       'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'PaymentMethod'],\n      dtype='object')] are in the [columns]"

In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 2311 entries, 0 to 2310
dtypes: float64(2311)
memory usage: 176.3 MB


In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# Let us define the response variable
y = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)



# Separate the categorical and numerical features from the original DataFrame
categorical_features = df[['County', 'Area', 'Job', 'Marital', 'Gender', 'Techie', 'Contract', 'Port_modem', 'Tablet', 'InternetService', 'Phone', 'Multiple', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'PaymentMethod']]
numeric_features = df[['Children', 'Age', 'Income', 'Outage_sec_perweek', 'Yearly_equip_failure', 'MonthlyCharge', 'Bandwidth_GB_Year']]

# Encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = encoder.fit_transform(categorical_features).toarray()

# Concatenate encoded categorical features with numerical features
X_final = np.concatenate([X_encoded, numeric_features.values], axis=1)
X = pd.DataFrame(X_final)

# Scale the features
X_scaled = StandardScaler().fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Fit the logistic regression model
model = sm.Logit(y_train, X_train).fit()
print(model.summary())


C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2441: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix